In [ ]:
import glob
import logging
import os
import sys

import numpy as np
import fitsio

import pyds9

%matplotlib notebook
import matplotlib
import matplotlib.pyplot as plt

import fpga
from fpga import geom

from testing import ccdProcedures


In [ ]:
%pdb off
d = pyds9.DS9() # '7f000101:45339')
d.set('view buttons no')
d.set('view layout vertical')

print(d)

From the .../LOG.txt files, find a run of biases to construct a superbias from. You only need to do this once per notebook run.

In [ ]:
reload(geom)
b1_biasNames = []
b1_biasNames.extend(glob.glob('/data/pfs/2016-08-15/PFJA00754*91.fits'))

b1_biasNames = []
b1_biasNames.extend(glob.glob('/data/pfs/2017-09-07/PFJA00925[3-7]*'))

biasExp = geom.clippedStack(b1_biasNames)

In [ ]:
reload(ccdProcedures)
print biasExp.image.dtype, biasExp.image.shape
_, _, ret = ccdProcedures.imStats(biasExp.image)
ccdProcedures.printStats(ret)

for i, n in enumerate(b1_biasNames):
    _, _, ret = ccdProcedures.imStats(n)
    ccdProcedures.printStats(ret)

In [ ]:
reload(geom)
exp = geom.Exposure('/data/pfs/2017-09-07/PFJA00925211.fits', dtype='i4', logLevel=5)
print exp.image.shape, exp.header['temps.CCD0']

In [ ]:
_, _, stats = ccdProcedures.imStats(exp)
ccdProcedures.printStats(stats)

In [ ]:
exp.header

In [ ]:
reload(geom)

d.set('frame delete all')
d.set('tile grid')
d.set('zoom 1')
d.set('scale zscale')

d.set('frame 1')
d.set_np2arr(biasExp.image)
print biasExp.image.dtype, biasExp.image.shape

fname = b1_biasNames[0] # '/data/pfs/2016-08-15/PFJA00756091.fits'
# fname = '/data/pfs/2016-08-15/PFJA00752591.fits'
fexp = geom.Exposure(fname, dtype='i4')
d.set('frame 2')
d.set_np2arr(fexp.image)
print fexp.image.dtype, fexp.image.shape


ampImages, overscanImages, _ = fexp.splitImage()
for i in range(len(ampImages)):
    ampImages[i] -= int(np.median(ampImages[i]))
    overscanImages[i] -= int(np.median(overscanImages[i]))
    
d.set('frame 3')
d.set_np2arr(np.hstack(ampImages))
#d.set('scale zscale')

d.set('frame 4')
d.set_np2arr(np.hstack(overscanImages))
#d.set('scale zscale')



In [ ]:
%pdb off
reload(geom)

#fname = b1_biasNames[0] # '/data/pfs/2016-08-15/PFJA00756091.fits'
fname = '/data/pfs/2017-09-07/PFJA00927211.fits'
fexp = geom.Exposure(fname, dtype='i4')

_, _, ret = ccdProcedures.imStats(fname)
ccdProcedures.printStats(ret)

ampImages, overscanImages = fexp.biasSubtract(fexp)

for i in range(len(ampImages)):
    print "%d: %0.1f, %0.1f" % (i, np.median(ampImages[i]), np.median(overscanImages[i]))

d.set('frame delete all')
d.set('tile grid')
d.set('zoom 1')
d.set('scale zscale')

d.set('frame 1')
d.set_np2arr(np.hstack(ampImages))
#d.set('scale zscale')

d.set('frame 2')
d.set_np2arr(np.hstack(overscanImages))
#d.set('scale zscale')

d.set('frame 3')
d.set_np2arr(fexp.image)
#d.set('scale zscale')

#d.set('frame 4')
#d.set_np2arr(biasExp.image)
#print biasExp.image.dtype, fexp.image.dtype
#d.set('scale zscale')

d.set('frame 1')



In [ ]:
qeList = glob.glob('/data/pfs/2015-12-21/PFSA00722*92.fits')
qeList.extend(glob.glob('/data/pfs/2015-12-21/PFSA00721992.fits'))
qeList.sort()          

In [ ]:
def qeStats1(fname):
    _, _, ret = ccdProcedures.imStats(fname)
    ccdProcedures.printStats(ret)
    
    hdr = fitsio.read_header(fname)

    exptime = hdr['EXPTIME']
    wavelength = hdr['qe.wave']
    energy = hdr['qe.energy']
    flux = energy / exptime
    
    print ""
    return ret, hdr

In [ ]:
stats, hdr = qeStats1(qeList[3])

In [ ]:
reload(geom)
reload(ccdProcedures)

pfiles = []
pfiles.extend(glob.glob('/data/pfs/2015-12-03/PFSA00618*92.fits'))
pfiles.extend(glob.glob('/data/pfs/2015-12-03/PFSA00619[0-7]92.fits'))
pfiles.extend(glob.glob('/data/pfs/2015-12-03/PFSA00624[2-8]92.fits'))

# Low flats
lffiles0 = glob.glob('/data/pfs/2015-12-04/PFSA00632[4-7]92.fits')
lffiles0.extend(glob.glob('/data/pfs/2015-12-04/PFSA0063[3-7]*92.fits'))
lffiles0.extend(glob.glob('/data/pfs/2015-12-04/PFSA00638[0-8]92.fits'))

lffiles1 = glob.glob('/data/pfs/2015-12-04/PFSA00640992.fits')
lffiles1.extend(glob.glob('/data/pfs/2015-12-04/PFSA0064[1-8]*92.fits'))

lffiles2 = glob.glob('/data/pfs/2015-12-04/PFSA00650[1-9]92.fits')
lffiles2.extend(glob.glob('/data/pfs/2015-12-04/PFSA0065[1-7]*92.fits'))
    
allLflats = lffiles0 + lffiles1 + lffiles2 
allLflats.sort()
all2sFlats = [f for f in allLflats if fitsio.read_header(f)['EXPTIME'] == 2]

# Trip some noisier images
all2sFlats = all2sFlats[4:]

# Bright flats:
bffiles = glob.glob('/data/pfs/2015-12-03/PFSA00619[9]*92.fits')
bffiles.extend(glob.glob('/data/pfs/2015-12-03/PFSA0062[012]*92.fits'))
bffiles.extend(glob.glob('/data/pfs/2015-12-03/PFSA00623[0-2]*92.fits'))

pfiles = bffiles[:]
pfiles.sort()

flstats = []
for f in sbias8:
    _, _, stat1 = ccdProcedures.imStats(f)
    flstats.append(stat1[3])

In [ ]:
list([f['noise'] for f in flstats])

In [ ]:
_f, _pl = plt.subplots(nrows=5, figsize=(8,15))
fits = []
cm = plt.get_cmap('gist_rainbow')
colors = [cm(i/8.0) for i in range(8)]
print ""
for i in range(8):
    pos_w = (da['exptime'][i,0] > 0) & (da['exptime'][i,0] < 160)
    x = da['exptime'][pos_w,i]
    y = da['signal'][pos_w,i]
    x_ii = np.argsort(x)
    x = x[x_ii]
    y = y[x_ii]
    fit = np.polyfit(x,y,2)
    _pl[0].plot(x, y-np.polyval(fit, x), '+', color=colors[i])
    #_pl[0].plot(x, y-np.polyval(fit, x), '-', color=colors[i], alpha=0.1)
    _pl[0].set_ylabel('signal - fit line')
    _pl[0].set_xlabel('exposure time (0.25mm slit)')

    _pl[1].plot(x, y, '+', color=colors[i])
    _pl[1].plot(x, y, '-', color=colors[i], alpha=0.1)
    _pl[1].set_ylabel('signal')
    _pl[1].set_xlabel('exposure time (0.25mm slit)')

    bias_ii = np.argsort(da['bias'][i])
    bias0 = da[:,i,2][0]
    _pl[2].plot(da['bias'][i]-bias0, '+', color=colors[i])
    _pl[2].plot(da['bias'][i]-bias0, '-', color=colors[i], alpha=0.1)
    _pl[2].set_ylabel('overscan - overscan[exp#0]')
    _pl[2].set_xlabel('exposure #')

    pa0temps = da['preamptemp'][i]
    ccd0temps = da['ccd0temp'][i]
    _pl[3].plot(ccd0temps-ccd0temps[0], 'x')
    _pl[3].plot(ccd0temps-ccd0temps[0], '-', alpha=0.1)
    _pl[3].set_ylabel('ccd temp - temp[exp#0]')
    _pl[3].set_xlabel('exposure #')

    _pl[4].plot(pa0temps-pa0temps[0], '+')
    _pl[4].plot(pa0temps-pa0temps[0], '-', alpha=0.1)
    _pl[4].set_ylabel('preamp temp - temp[exp#0]')
    _pl[4].set_xlabel('exposure #')
    
    
_pl[0].set_title('%s to %s' % (os.path.split(pfiles[0])[-1],
                               os.path.split(pfiles[-1])[-1]))
_f.tight_layout()

In [ ]:
reload(geom)
reload(ccdProcedures)

ffiles = glob.glob('/data/pfs/2015-12-04/PFSA00673[56]92.fits')
ffiles.extend(glob.glob('/data/pfs/2015-12-04/PFSA00675[78]92.fits'))
#ffiles = all2sFlats
fbias = sbias7

colCTEs, rowCTEs, allNormedCols, allNormedOsCols, allNormedRows, allNormedOsRows = \
     ccdProcedures.CTEStats(ffiles, fbias)

In [ ]:
ff, pl = plt.subplots(nrows=5, ncols=1, figsize=(10,16))
for i in range(8):
    pl[0].plot(allNormedCols[i][:,-1])
    pl[0].set_title('last active col, %d %0.1fs flats' % (len(ffiles), 
                                                          fitsio.read_header(ffiles[0])['EXPTIME']))
    pl[1].plot(allNormedOsCols[i][:,0])
    pl[1].set_title('first overscan col')
    pl[2].plot(allNormedRows[i][-2,:])
    pl[2].set_title('last-2 active row')
    pl[3].plot(allNormedRows[i][-1,:])
    pl[3].set_title('last active row')
    pl[4].plot(allNormedOsRows[i][0,:])
    pl[4].set_title('first overscan row')
    
ff.tight_layout()

In [ ]:
ff.savefig('/home/pfs/CTElines-1k.pdf')

In [ ]:
pfiles = glob.glob('/data/pfs/2015-12-05/PFSA00*92.fits')
pfiles.extend(glob.glob('/data/pfs/2015-12-04/PFSA00658[3-9]92.fits'))
pfiles.extend(glob.glob('/data/pfs/2015-12-04/PFSA00659*92.fits'))
pfiles.extend(glob.glob('/data/pfs/2015-12-04/PFSA006[67]*92.fits'))
pfiles.sort()

dat = np.zeros((len(pfiles), 3+8*2))

for i, f in enumerate(pfiles):
    exp = geom.Exposure(f)
    im = exp.image
    hdr = exp.header
    exptime = hdr['EXPTIME']
    
    print "=========== %s %s %4d %0.2f" % (os.path.split(f)[-1],
                                           hdr['IMAGETYP'], exptime, 
                                           float(hdr['temp.ccd0']))
    
    ret = ccdProcedures.imStats(exp)
    geom.printStats(ret[-1])
    dat[i,0] = exptime
    dat[i,1] = float(hdr['temp.pa'])
    dat[i,2] = float(hdr['temp.ccd0'])
    dat[i,3:3+8] = ret[-1]['signal']
    dat[i,3+8:3+16] = ret[-1]['bias']
    print

In [ ]:
ff, pl = plt.subplots(nrows=3, figsize=(12,12))
linfits = []
imNum = np.arange(len(dat))
for i in range(8):
    w_bias = dat[:,0] == 0
    w_keep = dat[:,0] >= 0
    
    #x = d[w_keep,0]
    imId = imNum[w_keep]
    paTemp = dat[w_keep,1]
    ccdTemp = dat[w_keep,2]
    biasLevel = dat[w_keep,i+3+8]
    # linfits.append(np.polyfit(x, y, 2))
    # resid = y - np.polyval(linfits[-1], x)
    pl[0].plot(imId[w_bias], biasLevel[w_bias]-biasLevel[0], '+')
    pl[0].plot(imId[w_bias != True], biasLevel[w_bias != True]-biasLevel[0], 'x', color='r')
    pl[1].plot(imId[w_bias], paTemp[w_bias], '+')
    pl[1].plot(imId[w_bias!=True], paTemp[w_bias!=True], 'x', color='red')
    pl[2].plot(imId[w_bias], ccdTemp[w_bias], '+')
    pl[2].plot(imId[w_bias!=True], ccdTemp[w_bias!=True], 'x', color='red')
    
pl[0].set_ylabel('overscan - overscan[#0], cnts')
pl[0].set_title('groups of 20 biases + 2 flats, separated by 30 min (#220 by 6 hours).')
pl[1].set_ylabel('preamp temp, K')
pl[2].set_ylabel('ccd0 temp, K')
    
ff.tight_layout()


In [ ]:
ff.savefig('/home/pfs/bias_offsets.pdf')

In [ ]:
reload(geom)
#ret = geom.flatStats('/data/pfs/2015-12-03/PFSA00629792.fits', '/data/pfs/2015-12-03/PFSA00629892.fits')
flats = glob.glob('/data/pfs/2017-08-27/PFJA00913[2-9]*.fits')
flats.extend(glob.glob('/data/pfs/2017-08-27/PFJA00914*.fits'))
flats.extend(glob.glob('/data/pfs/2017-08-27/PFJA00915[0-2]*.fits'))
flats = sorted(flats)

flats = glob.glob('/data/pfs/2017-08-27/PFJA00904[7-9]*.fits')
flats.extend(glob.glob('/data/pfs/2017-08-27/PFJA0090[5-8]*.fits'))
flats.extend(glob.glob('/data/pfs/2017-08-27/PFJA00909[0-5]*.fits'))
flats = sorted(flats)

def sublists(l, n=2):
    for i in range(0, len(l) - (n-1), n):
        yield l[i:i+n]

fpairs = (('/data/pfs/2015-12-03/PFSA00624292.fits', '/data/pfs/2015-12-03/PFSA00624392.fits'),
          ('/data/pfs/2015-12-03/PFSA00619492.fits', '/data/pfs/2015-12-03/PFSA00619592.fits'),
          ('/data/pfs/2015-12-04/PFSA00664792.fits', '/data/pfs/2015-12-04/PFSA00664892.fits'),
          ('/data/pfs/2015-12-04/PFSA00669192.fits', '/data/pfs/2015-12-04/PFSA00669292.fits'),
          ('/data/pfs/2015-12-05/PFSA00680192.fits', '/data/pfs/2015-12-05/PFSA00680292.fits'),
          ('/data/pfs/2015-12-05/PFSA00695992.fits', '/data/pfs/2015-12-05/PFSA00696092.fits'),
         )

for fbias, fflat in sublists(flats, 2):
    
    hdr0 = fitsio.read_header(fbias)
    hdr1 = fitsio.read_header(fflat)
    
    if (hdr0['bias.CH0.OD'] != hdr1['bias.CH0.OD'] or
        hdr0['bias.CH0.OG'] != hdr1['bias.CH0.OG']):
        
        raise RuntimeError('voltages in pairs must match')
        
    if hdr0['IMAGETYP'].strip() != 'bias' or hdr1['IMAGETYP'].strip() != 'flat':
        raise RuntimeError('image pairs must be bias, flat (%s, %s)' %
                          (hdr0['IMAGETYP'], hdr1['IMAGETYP']))

    _, _, stats0 = ccdProcedures.imStats(fbias)
    _, _, stats1 = ccdProcedures.imStats(fflat)
    print "============ %s OD=%0.1f OG=%0.1f" % (fp0,
                                                 hdr1['bias.CH0.OD'],
                                                 hdr1['bias.CH0.OG'])
    ccdProcedures.printStats(stats0)
    ccdProcedures.printStats(stats1)
    print

In [ ]:

d.set('frame delete all')
d.set('tile grid')

d.set('frame 1')
d.set('scale zscale')
d.set_np2arr(sbias1)

ampIms, osIms = sbias1.splitImage()
ampIm = np.hstack([ai - np.median(ai) for ai in ampIms])
osIm = np.hstack([ai - np.median(ai) for ai in osIms])

d.set('frame 2')
d.set('scale zscale')
d.set_np2arr(ampIm)

d.set('frame 3')
d.set('scale zscale')
d.set_np2arr(osIm)


In [ ]:
fbias = sbias8
fexp =  geom.Exposure('/data/pfs/2015-12-04/PFSA00671392.fits').image
#fexp = fitsio.read(lastFile(imdir)).astype('i4')
fbs = fexp - fbias


#printAmpStats(fbs)
#printOverscanStats(fbs)

d.set('frame 1')
d.set('scale zscale')
d.set_np2arr(fbs)
d.set('frame 2')
d.set('scale zscale')
d.set_np2arr(fbias)
d.set('frame 3')
d.set('scale zscale')
d.set_np2arr(fexp)

d.set('frame frameno 1')

In [ ]:
imdir = '/data/pfs/2015-12-03'
def lastFile(imdir):
    pat = "%s/*.fits" % (imdir)
    fl = glob.glob(pat)
    print pat, len(fl)
    fl.sort()
    return fl[-1]

fname = lastFile(imdir)
#fname = '/data/pfs/2015-11-20/PFSA00554012.fits'
f, fhdr = fitsio.read(fname, header=True)
#f = np.copy(sbias)
f = f.astype('i4')
if False:
    hi_w = np.where(f > 54000)
    f[hi_w] -= 65535

print fname
print "OD: ", fhdr.get('bias.CH0.OD'), fhdr.get('bias.CH1.OD')
print "OG: ", fhdr.get('bias.CH0.OG'), fhdr.get('bias.CH1.OG')
print "temps: ", fhdr.get('temp.PA'), fhdr.get('temp.CCD0'), fhdr.get('temp.CCD1')

#f = fbs
figname = 'bitcnt'
_f = plt.figure(figname)
plt.close(figname)
del _f
_f,pl = plt.subplots(2,1,num=figname)

oses = []
amps = range(8)
nrows = f.shape[0]
means = []
osstds = []
ampstds = []

for i in amps:
    os1 = osImage(f, i)
    ampImg = ampImage(f,i)
    os1 = os1[2000:2100, 5:-2]
    oses.append(os1 - np.median(os1))
    pl[0].plot(bitcnt(os1), '-+')
    row = ampImg[nrows/2,20:-35]
    pl[1].plot(row - row.mean() + i*10)
    means.append(np.mean(os1))
    osstds.append(np.std(os1))
    ampstds.append(np.std(ampImg[2000:2100, 200:300]))

pl[1].set_ylim(-50, 100)

print np.round(means,2)
print np.round(osstds,2)
print np.round(ampstds,2)
osim = np.concatenate(oses, axis=1)
doDisplay = False
if doDisplay and d is not None:
    d.set('frame 4')
    d.set_np2arr(osim)
    d.set('frame 5')
    d.set_np2arr(f)

    d.set('frame frameno 5')
